# Intro

Input = an email from Stores, confirming that an order has been placed

Desired output = Excel spreadsheet with a line for each item from the order, with columns arranged exactly as in Michal's spreadsheet, so you can just copy-paste the whole line in there. (With some exceptions - eg what type of item it is - this will have to be done manually)

# Outline of steps

## 1. Read in Stores email

Ideally could just paste it dierectly into the script

## 2. Parse through the email & extract relevant bits


## 3. Create a dataframe line for each item


## 4. Save the dataframe as an Excel spreadsheet

In [1]:
import pandas as pd
import re

In [2]:
# Get user input - pasted text from Stores email

email = input("Please paste Stores email here:\n\n")

Please paste Stores email here:

Submitted on Thursday, 12 December, 2019 - 09:30 Submitted by user: Miss Kotryna Bloznelyte Submitted values are:  Name: Miss Kotryna Bloznelyte Telephone number: 766029 Email address: luisilab@hermes.cam.ac.uk Research group: B Luisi How are you paying for your order? Research grant (e.g. PHAB/123) Research grant code: PhZJ/564 Research grant task number: 1 Research grant end date: 2022-08-31 Is your order VAT exempt?  : Yes, internal order Destination: Sanger Building Quotation: No, not applicable Attachment: Supplier: Stores Is an alternative supplier permissible? Yes Currency: GBP Minimum charge: Yes       ------------------------------------------------------------------------------ -------- DETAILS OF GOODS ----------------------------------------------------  Number of items: 2     ==Item 1:==       Quantity: 1       Unit size: 500ml       Description: Tween 20, P1379-500ML       Catalogue number: P1379-500ML       Approximate price each: £ 11.98

In [3]:
# Set up an empty dataframe

excel_lines = pd.DataFrame(columns=['Date', 'Name','Product', 'Cat. No.', 'Suppliers',
                                   'Amount', 'Unit size', 'Unit Price', 'Total Price', 'Grant No.', 'Date ordered'])
excel_lines

# Default choice for who ordered the item - would need to manually edit this if it's not lab
name = 'lab'

# Extract date from the email
day = email.split('Submitted on')[1].split(',')[1].split()[0]
#print(day)
if int(day) <10:
    day_2 = '0' + day
else:
    day_2 = day

month_name = email.split('Submitted on')[1].split(',')[1].split()[1]
#print(month_name)

from time import strptime
month_no = strptime(month_name,'%B').tm_mon
#print(month_no)
if int(month_no) <10:
    month_2 = '0' + str(month_no)
else:
    month_2 = str(month_no)

year = email.split('Submitted on')[1].split(',')[2].split()[0]
#print(year)
year_2 = str(list(year)[2]) + str(list(year)[3])
#print(year_2)

date_full = year + '-' + month_2 + '-' + day_2
#print(date_full)

date_2 = day + '/' + str(month_no) + '/' + year_2
#print(date_2)

# Extract other general info from the email

grant = email.split('Research grant code: ')[1].split()[0]
#print(grant)

supplier = email.split('Supplier: ')[1].split('Is an alternative')[0]
#print(supplier)

number_of_items = email.count("==Item")
#print(number_of_items)

currency = email.split('Currency: ')[1].split('Minimum charge')[0]
#print(currency)


# Extract item-specific info from the email
# Populate dataframe with the extracted info

for i in range(1, number_of_items + 1):
    
    #print(i)
    item = email.split('==Item ')[i]
    #print(item_i)
    quantity = int(item.split('Quantity: ')[1].split()[0])
    #print(quantity)
    unit_size = item.split('Unit size: ')[1].split('Description: ')[0]
    #print(unit_size)
    description = item.split('Description: ')[1].split('Catalogue number:')[0]
    #print(description)
    cat_no = item.split('Catalogue number: ')[1].split('Approximate price each: ')[0]
    #print(cat_no)
    
    price_region = item.split('Approximate price each: ')[1]
    #print(price_region)
    non_letters = re.findall('[0-9,.;: ]', price_region)
    non_letters_joined = ''.join(non_letters)
    price = non_letters_joined.split()[0]
    #print(price)
    price_number = float(price)
    #print(price_number)

    
    
    total_price = price_number * quantity
    total_price_dec = '{:.2f}'.format(total_price)
    
    if currency == "GBP" or 'gbp':
        currency_price = '£' + ' ' + str(price)
        total_price_currency = '£' + ' ' + str(total_price_dec)
    else:
        currency_price = str(price)
        total_price_currency = str(total_price_dec)
    
    new_line = {'Date': date_2, 'Name': name, 'Product': description, 'Cat. No.': cat_no, 'Suppliers': supplier,
            'Amount': quantity, 'Unit size': unit_size, 'Unit Price': currency_price,
            'Total Price': total_price_currency, 'Grant No.': grant, 'Date ordered': date_2}
    
    excel_lines = excel_lines.append(new_line, ignore_index=True)
    print(excel_lines)
    

       Date Name                       Product            Cat. No. Suppliers  \
0  12/12/19  lab  Tween 20, P1379-500ML         P1379-500ML          Stores    

  Amount     Unit size Unit Price Total Price Grant No. Date ordered  
0      1  500ml           £ 11.98     £ 11.98  PhZJ/564     12/12/19  


In [4]:
# Print the current version of the dataframe
print(excel_lines)

       Date Name                       Product            Cat. No. Suppliers  \
0  12/12/19  lab  Tween 20, P1379-500ML         P1379-500ML          Stores    

  Amount     Unit size Unit Price Total Price Grant No. Date ordered  
0      1  500ml           £ 11.98     £ 11.98  PhZJ/564     12/12/19  


In [5]:
# append to an existing Excel spreadsheet
from openpyxl import load_workbook

book = load_workbook('Ongoing_stores_emails.xlsx')
writer = pd.ExcelWriter('Ongoing_stores_emails.xlsx', engine='openpyxl')
writer.book = book
writer.sheets = {ws.title: ws for ws in book.worksheets}

for sheetname in writer.sheets:
    excel_lines.to_excel(writer,sheet_name=sheetname, startrow=writer.sheets[sheetname].max_row, index = False,header= False)

writer.save()